In [ ]:
# https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/spatial-data-vector-shapefiles/geographic-vs-projected-coordinate-reference-systems-python/
#https://epsg.io/
#https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/spatial-data-vector-shapefiles/epsg-proj4-coordinate-reference-system-formats-python/
#https://spatialreference.org/ref/epsg/


### Instalando os pacotes necessários

In [ ]:
pip install geopandas==0.10.2

In [ ]:
pip install fiona

In [ ]:
pip install folium

In [ ]:
pip install matplotlib

In [ ]:
pip install mapclassify

## Importando as bibliotecas

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import fiona
import numpy as np

# Parte introdutória

## Carregando os dados

In [ ]:
# Esse enconding serviu para os dados no meu ambiente.
shp_sp = gpd.read_file('estado_sp.shp', encoding = 'windows-1252')

### Características do shapefile

In [ ]:
print(type(shp_sp))
print(len(shp_sp))
print(shp_sp.bounds)
print(shp_sp.info())
shp_sp.crs

In [ ]:
shp_sp.head()

In [ ]:
### Alterando o nome das colunas para facilitar a manipulação dos dados
shp_sp.rename(columns = {'NM_MUNICIP':'municipio', 'CD_GEOCMU': 'codigo'}, inplace = True)
shp_sp

### Acessando os dados

In [ ]:
shp_sp[shp_sp['municipio'] == 'SÃO PAULO']

In [ ]:
shp_sp['codigo']

In [ ]:
shp_sp['geometry']

## Plotando o mapa
### Veja que é muuuuuuuito mais rápido que o R. Na minha máquina, a plotagem no R demorou mais de 30s

In [ ]:
shp_sp.plot(figsize=(10,10))

### Usando o método explore para criar um mapa interativo

In [ ]:
shp_sp.explore()

# Manipulação dos dados

## Cálculo da área

### Veja que no R a library para calcular a área o faz, independentement do fato da unidade dos dados ser em graus ou metros, devido ao CRS. No python, temos que usar um CRS em metros, ou seja, a referência é a mesma, mas, em vez de lat e long vamos usar metros. Como nosso shapefile estava em lat/long usamos o 5880.

In [ ]:
#Mostra que a unidade do CRS é em graus (longitude e latitude) -> Vai precisar alterar para metros para calcular as áreas
shp_sp.crs

In [ ]:
#CRS retirados de https://epsg.io/?q=brazil
#Transforma para o CRS 5880 que está com unidade em metros
shp_sp_m = shp_sp.to_crs(5880)

In [ ]:
shp_sp_m.crs

### Note que as unidade dos eixos x e y do mapa, agora estão em metros. 

In [ ]:
shp_sp_m.plot(figsize=(5,5))

In [ ]:
shp_sp_m['area'] = shp_sp_m.area/1000000 #Cria um novo feature no dataframe, chamado area, com os valores calculados.
shp_sp_m
# Veja que o valor das áreas calculadas tem uma pequena diferença em relação ao R...

### Visualização interativa

In [ ]:
shp_sp_m.explore("area")

## E caso haja a necessidade da inserção de dados externos?

### Vamos usar a biblioteca pyreadr para ler arquivos do tipo RData no Python

In [ ]:
# pip install pyreadr
import pyreadr

In [ ]:
dadossp = pyreadr.read_r('dados_sp.RData')

In [ ]:
dadossp

In [ ]:
# Veja que o tipo de objeto é um dicionário ordenado.
type(dadossp)

In [ ]:
# Vamos obter a chave do dicionário para poder extrair os dados.
print(dadossp.keys())

In [ ]:
dados_sp = dadossp['dados_sp']

In [ ]:
dados_sp

In [ ]:
#O código do municipio estava era um número float e precisa ser transformado em str para poder ser utilizado com o shapefile.
dados_sp['codigo'] = dados_sp['codigo'].astype(int)
dados_sp['codigo'] = dados_sp['codigo'].astype(str)

## Fazendo o merge dos dois datasets, usando o codigo como index

In [ ]:
print(shp_sp_m.columns)
print(dados_sp.columns)

In [ ]:
shp_dados_sp = shp_sp_m.merge(dados_sp, how='inner', on='codigo')

In [ ]:
print(shp_dados_sp.shape)
shp_dados_sp.head()

In [ ]:
plt.hist(shp_dados_sp['idh'], bins = 'auto')
plt.title("IDH")

In [ ]:
shp_dados_sp['idh'].plot(kind = 'hist')

# Visualização de dados espaciais

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 8))
shp_dados_sp.plot(column='idh', scheme='QUANTILES', ax=ax,
        edgecolor='white', legend=True, linewidth=0.3)
ax.set_axis_off()
plt.title('A distribuição do IDH no estado de SP', fontsize = 20)

plt.show()

In [ ]:
shp_dados_sp.head()

### Gráfico interativo relacionado à alguma feature

In [ ]:
shp_dados_sp.explore('pib')

# Desmembrando shapefiles

In [ ]:
shp_mundo = gpd.read_file('mundo.shp')
#shp_sp = gpd.read_file('estado_sp.shp', encoding = 'windows-1252')

In [ ]:
shp_mundo.plot(figsize = (10,10))

In [ ]:
shp_mundo.head()

## Plotando somente a América do Sul

In [ ]:
#Criando um filtro
americadosul = shp_mundo['contnnt'] == 'South America'

In [ ]:
shp_mundo[americadosul].plot(figsize=(10,10))

# Combinando shapefiles

## Carregando os shapefiles necessários

In [ ]:
shp_argentina = gpd.read_file('argentina_shapefile.shp')
shp_brasil = gpd.read_file('brasil_shapefile.shp')
shp_paraguai = gpd.read_file('paraguai_shapefile.shp')
shp_venezuela = gpd.read_file('venezuela_shapefile.shp')

In [ ]:
shp_mercosul = gpd.GeoDataFrame(pd.concat([shp_argentina, shp_brasil, shp_paraguai, shp_venezuela]))

In [ ]:
shp_mercosul

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 8))
shp_mercosul.plot(column='mercosul', ax=ax,
        edgecolor='white', legend=True, linewidth=0.3)
ax.set_axis_off()
plt.title('Membresia no Mercosul', fontsize = 20)

plt.show()